In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns; sns.set()  # for plot styling


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

from sklearn.cluster import MeanShift, estimate_bandwidth

from itertools import cycle
from sklearn.mixture import GaussianMixture

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
# Regresion lineal
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse

# q-q plots
import scipy.stats as stats

import statsmodels.api as sm

import statsmodels.formula.api as smf

from pandas_profiling import ProfileReport

# Para preparar los datos
from sklearn.preprocessing import LabelEncoder

# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 

# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier

# Para usar Random Tree Forest como clasificador
from sklearn.ensemble import RandomForestClassifier

# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import model_selection

# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Para la validación cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

#Librerías para la visualización
import matplotlib.pyplot as plt

# Seaborn
import seaborn as sns 
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

#SMF
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

# check version number
import imblearn

from collections import Counter

from imblearn.over_sampling import SMOTE, ADASYN

In [2]:
# Se cargan los datos
# Ubicación de los datos, deben estar en la misma carpeta que este notebook
pais = "espana"
liga = "laliga"
version = 0
db_location = f"data/{pais}_{liga}/version{version}/resultados_anteriores.xlsx"
db_location2 = f"data/{pais}_{liga}/version{version}/proximos_partidos.xlsx"

# db_location= "data_matches_mismarcadores_half0.xlsx"


# Se leen los datos, estan separados con ','
# data1 = pd.read_excel(db_location, sheet_name="Premier League 20222023")
# data2 = pd.read_excel(db_location, sheet_name="Premier League 20212022")
# data = pd.concat([data1,data2])
data = pd.read_excel(db_location)

proximos_partidos= pd.read_excel(db_location2)

In [3]:
data.fillna(value=0,inplace=True)
data

,Date,HomeTeam,AwayTeam,HG,AG,HP,AP,HTS,ATS,HSI,...,ATP,HPC,APC,HT,AT,HA,AA,HDA,ADA,Resultado
0,25.04.2023 15:00,Real Betis,Real Sociedad,0,0,58%,42%,10,10,1,...,337,354,245,19,19,118,110,43,58,0
1,25.04.2023 12:30,Cádiz,Osasuna,0,1,56%,44%,20,6,4,...,374,368,282,11,13,114,95,69,27,-1
2,25.04.2023 12:30,Girona,Real Madrid,4,2,28%,72%,13,18,5,...,706,219,643,26,15,66,133,30,87,2
3,23.04.2023 14:00,Sevilla,Villarreal,2,1,38%,62%,17,17,4,...,493,208,398,12,12,102,103,44,36,1
4,23.04.2023 11:30,Mallorca,Getafe,3,1,49%,51%,8,3,4,...,423,287,306,11,13,122,120,34,23,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,14.08.2022 10:30,Cádiz,Real Sociedad,0,1,40%,60%,7,15,2,...,461,218,374,20,25,83,104,44,55,-1
299,13.08.2022 14:00,Barcelona,Rayo Vallecano,0,0,67%,33%,21,4,6,...,274,489,191,16,18,116,78,64,24,0
300,13.08.2022 12:00,Real Valladolid,Villarreal,0,3,52%,48%,12,10,6,...,413,362,322,16,18,91,109,49,50,-3
301,13.08.2022 10:00,Celta de Vigo,Espanyol,2,2,59%,41%,8,10,3,...,357,442,276,19,18,116,127,31,34,0


In [4]:
def conseguir_anio(texto):
    return (texto.split()[0]).split(".")[2]
def conseguir_mes(texto):
    return (texto.split()[0]).split(".")[1]
def conseguir_dia(texto):
    return (texto.split()[0]).split(".")[0]
def quitar_por(texto):
    return (texto.replace("%",""))    

In [5]:
data['year'] = (data['Date'].apply(conseguir_anio))
data['month'] = (data['Date'].apply(conseguir_mes))
data['day'] = (data['Date'].apply(conseguir_dia))
data['HP'] = data['HP'].apply(quitar_por)
data['AP'] = data['AP'].apply(quitar_por)
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data['day'] = data['day'].astype(int)
data['HP'] = data['HP'].astype(int)
data['AP'] = data['AP'].astype(int)
data['Fecha']= pd.to_datetime(data[['year','month','day']])

data['TG'] = data['HG'] + data['AG']
# data['Mas2Goles'] = np.where(data['TG'] > 2,1,0)

data_backup= data.copy()

In [6]:
data_backup['Resultado_y'] = np.where(data_backup['Resultado']>=1,1,np.where(data_backup['Resultado']==0,0,-1))

In [7]:
data_backup['TG'].value_counts()

2    74
1    70
3    69
4    36
0    23
5    21
6     6
7     2
8     2
Name: TG, dtype: int64

In [8]:
data_backup_2 = data_backup[(data_backup['TG']>0) & (data_backup['TG']<6)].copy()

In [9]:
data_backup_2['TG'].value_counts()

2    74
1    70
3    69
4    36
5    21
Name: TG, dtype: int64

In [10]:
selected_cols = ['HTS', 'ATS', 'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HGS', 'AGS']

# Proceso de datos Goles

In [11]:
x_drop=data_backup_2[selected_cols]
feature_names= selected_cols
X_gol = np.array(x_drop)
y_gol = np.array(data_backup_2['TG'])

In [12]:
oversample_gol = SMOTE()
X_gol, y_gol = oversample_gol.fit_resample(X_gol, y_gol)

In [13]:
ss_gol = StandardScaler()
X_gol = ss_gol.fit_transform(X_gol)

# Proceso de datos Resultado_y

## Selección sub-datasets

In [14]:
x_drop=data_backup[selected_cols]
feature_names= selected_cols
X_resul = np.array(x_drop)
y_resul = np.array(data_backup['Resultado_y'])

## SMOTE

In [15]:
oversample_resul = SMOTE()
X_resul, y_resul = oversample_resul.fit_resample(X_resul, y_resul)

## Estandarización

In [16]:
ss_resul = StandardScaler()
X_resul = ss_resul.fit_transform(X_resul)

# Predicción Próximos Partidos

In [17]:
X_predicc= X_resul.copy()
y_predicc= y_resul.copy()

model_resul=LogisticRegression(random_state=1001)

model_resul.fit(X_predicc, y_predicc)

LogisticRegression(random_state=1001)

In [18]:
X_predicc_gol= X_gol.copy()
y_predicc_gol= y_gol.copy()

model_gol=LinearDiscriminantAnalysis()

model_gol.fit(X_predicc_gol, y_predicc_gol)

LinearDiscriminantAnalysis()

In [19]:
proximos_partidos = proximos_partidos[['Date','HomeTeam','AwayTeam']]

proximos_partidos['year'] = (proximos_partidos['Date'].apply(conseguir_anio))
proximos_partidos['month'] = (proximos_partidos['Date'].apply(conseguir_mes))
proximos_partidos['day'] = (proximos_partidos['Date'].apply(conseguir_dia))
proximos_partidos['year'] = proximos_partidos['year'].astype(int)
proximos_partidos['month'] = proximos_partidos['month'].astype(int)
proximos_partidos['day'] = proximos_partidos['day'].astype(int)
proximos_partidos['Fecha']= pd.to_datetime(proximos_partidos[['year','month','day']])

In [20]:
data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'HG', 'AG', 'HP', 'AP', 'HTS', 'ATS',
       'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HFK', 'AFK', 'HC', 'AC',
       'HOFF', 'AOFF', 'HTI', 'ATI', 'HGS', 'AGS', 'HF', 'AF', 'HRC', 'ARC',
       'HYC', 'AYC', 'HTP', 'ATP', 'HPC', 'APC', 'HT', 'AT', 'HA', 'AA', 'HDA',
       'ADA', 'Resultado', 'year', 'month', 'day', 'Fecha', 'TG'],
      dtype='object')

In [21]:
def calcularDiferenciaGol5Prom_pred():
    dataNueva= []
    for i in range(len(proximos_partidos)):
        infoAct=[]
        homeTeam = proximos_partidos.iloc[i]['HomeTeam']
        awayTeam = proximos_partidos.iloc[i]['AwayTeam']
        fecha_match =  proximos_partidos.iloc[i]['Fecha']
        infoHome = data[(data['HomeTeam'] == homeTeam) &  (data['Fecha'] < fecha_match )].copy().head(5)
        infoAway = data[(data['AwayTeam'] == awayTeam) & (data['Fecha'] < fecha_match )].copy().head(5)
        infoHome.drop(['Resultado', 'year','month', 'day','Fecha','TG'], axis=1,inplace=True)
        infoAway.drop(['Resultado', 'year','month', 'day','Fecha','TG'], axis=1,inplace=True)
        ##Solo se va a tomar en cuenta la información de cada equipo , más no las estadisticas que recibe en contra.
        infoHome = infoHome.mean()
        infoAway = infoAway.mean()
        if len(infoHome) > 38:
            for i in range(40):
                infoAct.append(0)
            dataNueva.append(infoAct)
            continue
        for i in range(0,len(infoHome),2):
            infoAct.append(infoHome[i])
            infoAct.append(infoAway[i+1])
        dataNueva.append(infoAct)
    return dataNueva

In [22]:
datosProm5Partidos_prox = calcularDiferenciaGol5Prom_pred()
cols= ["Prom5_HG","Prom5_AG","Prom5_HP","Prom5_AP","Prom5_HTS","Prom5_ATS","Prom5_HSI","Prom5_ASI","Prom5_HSO","Prom5_ASO","Prom5_HBS","Prom5_ABS","Prom5_HFK","Prom5_AFK","Prom5_HC","Prom5_AC","Prom5_HOF","Prom5_AOF","Prom5_HTI","Prom5_ATI","Prom5_HGS","Prom5_AGS","Prom5_HF","Prom5_AF","Prom5_HRC","Prom5_ARC","Prom5_HYC","Prom5_AYC","Prom5_HTP","Prom5_ATP","Prom5_HPC","Prom5_APC","Prom5_HT","Prom5_AT","Prom5_HA","Prom5_AA","Prom5_HDA","Prom5_ADA"]


In [23]:
dataProm5P_prox= pd.DataFrame(np.array(datosProm5Partidos_prox), columns=cols)
dataProm5P_prox

,Prom5_HG,Prom5_AG,Prom5_HP,Prom5_AP,Prom5_HTS,Prom5_ATS,Prom5_HSI,Prom5_ASI,Prom5_HSO,Prom5_ASO,...,Prom5_HTP,Prom5_ATP,Prom5_HPC,Prom5_APC,Prom5_HT,Prom5_AT,Prom5_HA,Prom5_AA,Prom5_HDA,Prom5_ADA
0,1.2,0.8,52.2,49.2,16.2,10.6,4.8,2.6,7.2,6.2,...,445.0,365.2,347.8,274.6,15.8,11.8,119.8,101.6,48.4,40.8
1,1.0,0.6,57.4,46.2,13.0,9.4,3.6,2.2,6.0,5.0,...,496.2,365.2,410.8,282.8,16.4,23.2,109.2,97.2,51.2,36.8
2,1.2,0.8,50.8,53.8,16.8,10.8,4.6,3.2,7.0,4.2,...,420.6,442.8,336.8,360.4,15.8,17.6,117.8,88.2,68.0,44.8
3,1.2,1.0,49.2,44.0,11.6,8.8,5.4,2.6,3.8,3.4,...,425.2,338.8,352.4,250.8,18.2,16.8,94.6,97.0,47.4,35.8
4,1.4,1.0,45.6,57.0,10.6,13.4,3.4,3.8,4.0,5.6,...,333.2,551.8,229.6,485.4,18.4,14.2,106.8,110.6,46.2,59.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.8,0.6,50.8,47.8,10.4,9.2,3.8,2.4,4.4,4.6,...,435.6,381.8,359.4,297.6,18.2,17.4,100.4,98.4,44.2,48.4
96,2.8,1.4,61.2,47.6,20.8,13.0,8.0,4.2,8.6,6.0,...,594.6,430.4,532.8,334.8,15.6,14.6,118.4,116.0,69.4,48.6
97,1.4,1.2,62.0,49.6,11.8,8.6,3.4,3.0,5.8,4.4,...,519.2,428.0,420.8,341.2,15.8,16.4,128.4,108.2,49.4,41.2
98,1.4,0.8,48.6,41.8,14.6,8.4,5.0,3.8,6.2,2.0,...,380.8,339.2,300.4,229.0,14.6,16.0,100.4,113.2,42.6,38.6


In [24]:
proximos_partidos= proximos_partidos.reset_index(drop=True)
dataProm5P_prox = dataProm5P_prox.reset_index(drop=True)

In [25]:
proximos_partidos

,Date,HomeTeam,AwayTeam,year,month,day,Fecha
0,14.04.2023 14:00,Rayo Vallecano,Osasuna,2023,4,14,2023-04-14
1,15.04.2023 07:00,Villarreal,Real Valladolid,2023,4,15,2023-04-15
2,15.04.2023 09:15,Athletic Club,Real Sociedad,2023,4,15,2023-04-15
3,15.04.2023 11:30,Real Betis,Espanyol,2023,4,15,2023-04-15
4,15.04.2023 14:00,Cádiz,Real Madrid,2023,4,15,2023-04-15
...,...,...,...,...,...,...,...
95,04.06.2023 11:30,Real Betis,Valencia,2023,6,4,2023-06-04
96,04.06.2023 11:30,Real Madrid,Athletic Club,2023,6,4,2023-06-04
97,04.06.2023 11:30,Real Sociedad,Sevilla,2023,6,4,2023-06-04
98,04.06.2023 11:30,Real Valladolid,Getafe,2023,6,4,2023-06-04


In [26]:
dataProm5P_mod_prox= pd.concat([proximos_partidos, dataProm5P_prox], axis=1)

In [27]:
dataProm5P_mod_prox

,Date,HomeTeam,AwayTeam,year,month,day,Fecha,Prom5_HG,Prom5_AG,Prom5_HP,...,Prom5_HTP,Prom5_ATP,Prom5_HPC,Prom5_APC,Prom5_HT,Prom5_AT,Prom5_HA,Prom5_AA,Prom5_HDA,Prom5_ADA
0,14.04.2023 14:00,Rayo Vallecano,Osasuna,2023,4,14,2023-04-14,1.2,0.8,52.2,...,445.0,365.2,347.8,274.6,15.8,11.8,119.8,101.6,48.4,40.8
1,15.04.2023 07:00,Villarreal,Real Valladolid,2023,4,15,2023-04-15,1.0,0.6,57.4,...,496.2,365.2,410.8,282.8,16.4,23.2,109.2,97.2,51.2,36.8
2,15.04.2023 09:15,Athletic Club,Real Sociedad,2023,4,15,2023-04-15,1.2,0.8,50.8,...,420.6,442.8,336.8,360.4,15.8,17.6,117.8,88.2,68.0,44.8
3,15.04.2023 11:30,Real Betis,Espanyol,2023,4,15,2023-04-15,1.2,1.0,49.2,...,425.2,338.8,352.4,250.8,18.2,16.8,94.6,97.0,47.4,35.8
4,15.04.2023 14:00,Cádiz,Real Madrid,2023,4,15,2023-04-15,1.4,1.0,45.6,...,333.2,551.8,229.6,485.4,18.4,14.2,106.8,110.6,46.2,59.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,04.06.2023 11:30,Real Betis,Valencia,2023,6,4,2023-06-04,0.8,0.6,50.8,...,435.6,381.8,359.4,297.6,18.2,17.4,100.4,98.4,44.2,48.4
96,04.06.2023 11:30,Real Madrid,Athletic Club,2023,6,4,2023-06-04,2.8,1.4,61.2,...,594.6,430.4,532.8,334.8,15.6,14.6,118.4,116.0,69.4,48.6
97,04.06.2023 11:30,Real Sociedad,Sevilla,2023,6,4,2023-06-04,1.4,1.2,62.0,...,519.2,428.0,420.8,341.2,15.8,16.4,128.4,108.2,49.4,41.2
98,04.06.2023 11:30,Real Valladolid,Getafe,2023,6,4,2023-06-04,1.4,0.8,48.6,...,380.8,339.2,300.4,229.0,14.6,16.0,100.4,113.2,42.6,38.6


In [28]:
def generar_predicciones_pred():
    predicciones_partido=[]
    predicciones_gol=[]
    proba_local=[]
    proba_empate=[]
    proba_visitante=[]
    for i in range(len(dataProm5P_mod_prox)):
        x_int= [dataProm5P_mod_prox.iloc[i]['Prom5_HTS'],dataProm5P_mod_prox.iloc[i]['Prom5_ATS'],
        dataProm5P_mod_prox.iloc[i]['Prom5_HSI'],dataProm5P_mod_prox.iloc[i]['Prom5_ASI'],dataProm5P_mod_prox.iloc[i]['Prom5_HSO'],dataProm5P_mod_prox.iloc[i]['Prom5_ASO'],dataProm5P_mod_prox.iloc[i]['Prom5_HBS'],
        dataProm5P_mod_prox.iloc[i]['Prom5_ABS'],dataProm5P_mod_prox.iloc[i]['Prom5_HGS'], dataProm5P_mod_prox.iloc[i]['Prom5_AGS'] ]
        x_int= np.array(x_int)
        # Estandarizacion 
        x_int_estd = ss_resul.transform([x_int])
        x_int_estd_gol = ss_gol.transform([x_int])
        feature_names_act= ['HTS', 'ATS', 'HSI', 'ASI', 'HSO', 'ASO', 'HBS', 'ABS', 'HGS', 'AGS']

        x_int_estd = pd.DataFrame(x_int_estd,columns=feature_names_act)
        x_int_estd= np.array(x_int_estd)

        x_int_estd_gol = pd.DataFrame(x_int_estd_gol,columns=feature_names_act)
        x_int_estd_gol= np.array(x_int_estd_gol)

        predicc = model_resul.predict(x_int_estd)
        predicc_gol = model_gol.predict(x_int_estd_gol)

        predicc_proba= model_resul.predict_proba(x_int_estd)
        proba_local.append((predicc_proba[0])[2])
        proba_empate.append((predicc_proba[0])[1])
        proba_visitante.append((predicc_proba[0])[0])
        predicciones_partido.append(predicc)
        predicciones_gol.append(predicc_gol)
    return [predicciones_partido,predicciones_gol,proba_local,proba_empate,proba_visitante]

In [29]:
resultados= generar_predicciones_pred()
predicciones_partido= resultados[0]
predicciones_gol = resultados[1]
proba_local= resultados[2]
proba_empate= resultados[3]
proba_visitante= resultados[4]
dataProm5P_mod_prox['Resultado_predicc']= pd.Series(predicciones_partido)
dataProm5P_mod_prox['Resultado_predicc_mod']= np.where(dataProm5P_mod_prox['Resultado_predicc']==1,"Local",np.where(dataProm5P_mod_prox['Resultado_predicc']==0,"Empate","Visitante"))
dataProm5P_mod_prox['Prediccion_gol']= pd.Series(predicciones_gol)
dataProm5P_mod_prox['Prediccion_gol_apuesta'] = np.where(((dataProm5P_mod_prox['Prediccion_gol']>=3)),"Hándicap +2 Goles o +1.5 Goles","Hándicap -4 Goles o -3.5 Goles")
dataProm5P_mod_prox['proba_local']= pd.Series(proba_local)
dataProm5P_mod_prox['proba_empate']= pd.Series(proba_empate)
dataProm5P_mod_prox['proba_visitante']= pd.Series(proba_visitante)

In [30]:
pd.options.display.max_rows = 500

In [31]:
 dataProm5P_mod_prox[['Date','HomeTeam','AwayTeam','Prediccion_gol_apuesta','Prediccion_gol','Resultado_predicc_mod','proba_local','proba_empate','proba_visitante']]

,Date,HomeTeam,AwayTeam,Prediccion_gol_apuesta,Prediccion_gol,Resultado_predicc_mod,proba_local,proba_empate,proba_visitante
0,14.04.2023 14:00,Rayo Vallecano,Osasuna,Hándicap -4 Goles o -3.5 Goles,[1],Local,6.616993e-01,0.328003,1.029807e-02
1,15.04.2023 07:00,Villarreal,Real Valladolid,Hándicap -4 Goles o -3.5 Goles,[1],Visitante,7.360455e-05,0.058165,9.417609e-01
2,15.04.2023 09:15,Athletic Club,Real Sociedad,Hándicap +2 Goles o +1.5 Goles,[3],Empate,1.167893e-01,0.491638,3.915731e-01
3,15.04.2023 11:30,Real Betis,Espanyol,Hándicap -4 Goles o -3.5 Goles,[2],Local,9.274563e-01,0.071377,1.167172e-03
4,15.04.2023 14:00,Cádiz,Real Madrid,Hándicap -4 Goles o -3.5 Goles,[1],Visitante,6.675284e-02,0.461554,4.716929e-01
5,16.04.2023 07:00,Girona,Elche,Hándicap -4 Goles o -3.5 Goles,[1],Local,5.344085e-01,0.426990,3.860192e-02
6,16.04.2023 09:15,Getafe,Barcelona,Hándicap -4 Goles o -3.5 Goles,[2],Local,9.027930e-01,0.095476,1.731243e-03
7,16.04.2023 11:30,Atlético de Madrid,Almería,Hándicap +2 Goles o +1.5 Goles,[5],Visitante,1.530252e-02,0.152290,8.324070e-01
8,16.04.2023 14:00,Valencia,Sevilla,Hándicap +2 Goles o +1.5 Goles,[3],Local,5.061310e-01,0.428015,6.585420e-02
9,17.04.2023 14:00,Celta de Vigo,Mallorca,Hándicap +2 Goles o +1.5 Goles,[3],Local,6.015726e-01,0.365429,3.299817e-02


In [32]:
pd.options.display.max_rows = 20


# TODO: PROBAR CON PROMEDIO DE 10 PARTIDOS MÁXIMO Y CON PROMEDIO SIN DISCRMINAR LOCALIA O VISTA
#TODO: TUNEAR LOS MODELOS

# <a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ed14a935-a2d1-49c6-bc5b-8778463ac475' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>